In [2]:
import os

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

## Loading LendingClub Data

It is expected that the accepted and rejected loan datasets live in a subdirectory called ```data``` in the root directory.

In [20]:
DATASET_PATH = os.path.join(os.path.dirname(os.getcwd()), 'data/')
ACCEPTED_LOANS_FN = "Lending_Club_Accepted_2014_2018.csv"
REJECTED_LOANS_FN = "Lending_Club_Rejected_2014_2018.csv"

accepted = pd.read_csv(DATASET_PATH + ACCEPTED_LOANS_FN)
rejected = pd.read_csv(DATASET_PATH + REJECTED_LOANS_FN)

/home/acowlagi/.conda/envs/torch_py39/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (19,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Exploring the Accepted Loans Dataset

In [21]:
accepted.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.00,0.00,25679.660000,25679.66,24700.00,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN,N

### Subsampling By Loan Amount

For the sake of initial exploration, we sample from the accepted loans data to construct a representative subset of the data. We do this by sampling from a discrete approximation of the distribution of ```loan_amnt``` category to apply a 50x reduction in the dataset size.

In [23]:
TEST_PATH = os.path.join(os.path.dirname(os.getcwd()), 'rawDataProcessing/test_files/')
SAMPLE_BY_LOAN_FN = "sample_by_loan_amt.csv"
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 160)

In [25]:
sampled_by_loan = pd.read_csv(TEST_PATH + SAMPLE_BY_LOAN_FN)
sampled_by_loan.head()

/home/acowlagi/.conda/envs/torch_py39/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (19,59,118) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68533595,NaN,12000.0,12000.0,12000.0,36 months,10.78,391.62,B,B4,Public Affairs Specialist,1 year,RENT,98000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,993xx,WA,24.04,0.0,Feb-2002,690.0,694.0,0.0,33.0,61.0,15.0,1.0,20462.0,62.8,39.0,w,0.0,0.0,13349.665589,13349.67,12000.0,1349.67,0.0,0.00,0.0000,Apr-2017,7884.96,NaN,Jun-2017,754.0,750.0,0.0,69.0,1.0,Individual,NaN,NaN,NaN,0.0,5930.0,374585.0,0.0,4.0,1.0,3.0,9.0,82819.0,66.0,1.0,2.0,12597.0,65.0,32600.0,1.0,1.0,0.0,5.0,26756.0,3884.0,83.9,0.0,0.0,147.0,166.0,9.0,9.0,8.0,9.0,69.0,15.0,69.0,4.0,3.0,5.0,3.0,6.0,17.0,8.0,14.0,5.0,15.0,0.0,0.0,0.0,2.0,87.2,66.7,0.0,0.0,478544.0,103281.0,24100.0,120544.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68495092,NaN,8650.0,8650.0,8650.0,36 months,19.89,320.99,E,E3,Program Coordinator,8 years,RENT,55000.0,Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,462xx,IN,25.49,0.0,Mar-2005,675.0,679.0,4.0,NaN,30.0,18.0,1.0,9568.0,46.0,19.0,w,0.0,0.0,9190.490000,9190.49,8650.0,540.49,0.0,0.00,0.0000,May-2016,8251.42,NaN,Jun-2016,639.0,635.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,18926.0,6.0,1.0,0.0,1.0,19.0,9358.0,51.0,12.0,16.0,653.0,46.0,20750.0,2.0,0.0,5.0,17.0,1051.0,1375.0,45.0,0.0,0.0,129.0,95.0,0.0,0.0,0.0,8.0,NaN,0.0,NaN,0.0,2.0,17.0,2.0,2.0,2.0,17.0,17.0,13.0,18.0,NaN,0.0,0.0,12.0,100.0,50.0,1.0,0.0,38998.

We now explore the structure of various features of initial interest. We drop the remaining columns for this section. In general, we have extracted features that are known prior to/immediately after loan origination, rather than at the end of the lifetime of the loan. We also include the loan status. 

In [29]:
cat_vars_of_interest = ["term", "grade", "sub_grade", "emp_title", "emp_length", "home_ownership", "verification_status", "purpose", "desc", "addr_state", 
    "initial_list_status", "policy_code", "application_type", "hardship_flag", "hardship_type", "hardship_reason",  "loan_status",
]
num_vars_of_interest = [
    "loan_amnt", "funded_amnt", "funded_amnt_inv", "int_rate", "installment", "annual_inc", "dti", "delinq_2yrs", "earliest_cr_line", "fico_range_low", "fico_range_high", 
    "revol_bal", "revol_util", "open_acc", "annual_inc", "issue_d", "zip_code", "delinq_2yrs", "inq_last_6mths", "open_acc", "total_acc",  
    "last_fico_range_low", "last_fico_range_high", "open_il_12m", "open_il_24m", "total_bal_il", "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc", "all_util",
    "total_rev_hi_lim", "inq_fi", "total_cu_tl", "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy", "bc_util", "num_accts_ever_120_pd",
    "num_actv_bc_tl", "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_sats", "num_il_tl", "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0", "tot_hi_cred_lim", 
    "pct_tl_nvr_dlq", "percent_bc_gt_75", "total_bal_ex_mort", "total_bc_limit","total_il_high_credit_limit"
]


In [30]:
sampled_by_loan = sampled_by_loan[num_vars_of_interest + cat_vars_of_interest]
sampled_by_loan.head()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,annual_inc,dti,delinq_2yrs,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,revol_bal,revol_util,open_acc,open_acc,annual_inc,annual_inc,issue_d,zip_code,delinq_2yrs,delinq_2yrs,inq_last_6mths,open_acc,open_acc,total_acc,last_fico_range_low,last_fico_range_high,open_il_12m,open_il_24m,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_sats,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,tot_hi_cred_lim,pct_tl_nvr_dlq,percent_bc_gt_75,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,term,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,purpose,desc,addr_state,initial_list_status,policy_code,application_type,hardship_flag,hardship_type,hardship_reason,loan_status
0,12000.0,12000.0,12000.0,10.78,391.62,98000.0,98000.0,24.04,0.0,0.0,Feb-2002,690.0,694.0,20462.0,62.8,15.0,15.0,98000.0,98000.0,Dec-2015,993xx,0.0,0.0,0.0,15.0,15.0,39.0,750.0,754.0,1.0,3.0,82819.0,66.0,1.0,2.0,12597.0,65.0,32600.0,1.0,1.0,0.0,5.0,26756.0,3884.0,83.9,4.0,3.0,5.0,3.0,6.0,15.0,17.0,8.0,14.0,5.0,478544.0,87.2,66.7,103281.0,24100.0,120544.0,36 months,B,B4,Public Affairs Specialist,1 year,RENT,Not Verified,debt_consolidation,NaN,WA,w,1.0,Individual,N,NaN,NaN,Fully Paid
1,8650.0,8650.0,8650.0,19.89,320.99,55000.0,55000.0,25.49,0.0,0.0,Mar-2005,675.0,679.0,9568.0,46.0,18.0,18.0,55000.0,55000.0,Dec-2015,462xx,0.0,0.0,4.0,18.0,18.0,19.0,635.0,639.0,0.0,1.0,9358.0,51.0,12.0,16.0,653.0,46.0,20750.0,2.0,0.0,5.0,17.0,1051.0,1375.0,45.0,0.0,2.0,17.0,2.0,2.0,18.0,2.0,17.0,17.0,13.0,38998.0,100.0,50.0,18926.0,2750.0,18248.0,36 months,E,E3,Program Coordinator,8 years,RENT,Verified,debt_consolidation,NaN,IN,w,1.0,Individual,N,NaN,NaN,Fully Paid
2,25000.0,25000.0,25000.0,7.49,777.55,109000.0,109000.0,26.02,0.0,0.0,Dec-2001,745.0,749.0,20862.0,54.3,9.0,9.0,109000.0,109000.0,Dec-2015,226xx,0.0,0.0,1.0,9.0,9.0,19.0,720.0,724.0,0.0,1.0,47194.0,58.0,0.0,1.0,8937.0,57.0,38400.0,1.0,0.0,1.0,2.0,33976.0,17538.0,54.3,0.0,3.0,3.0,5.0,6.0,9.0,7.0,5.0,9.0,3.0,373572.0,100.0,20.0,68056.0,38400.0,82117.0,36 months,A,A4,Sales Manager,10+ years,MORTGAGE,Not Verified,debt_consolidation,NaN,VA,w,1.0,Individual,N,NaN,NaN,Fully Paid
3,4225.0,4225.0,4225.0,14.85,146.16,35000.0,35000.0,15.22,2.0,2.0,Jul-2011,725.0,729.0,1058.0,24.6,6.0,6.0,35000.0,35000.0,Dec-2015,672xx,2.0,2.0,0.0,6.0,6.0,6.0,0.0,499.0,0.0,0.0,3830.0,22.0,0.0,0.0,367.0,22.0,4300.0,0.0,0.0,0.0,0.0,815.0,2233.0,14.1,0.0,1.0,2.0,3.0,3.0,6.0,1.0,5.0,5.0,2.0,21822.0,66.7,0.0,4888.0,2600.0,17522.0,36 months,C,C5,mechanic,5 years,RENT,Source Verified,debt_consolidation,NaN,KS,w,1.0,Individual,N,NaN,NaN,Charged Off
4,1400.0,1400.0,1400.0,12.88,47.10,64000.0,64000.0,34.95,0.0,0.0,Jun-1996,700.0,704.0,37828.0,67.2,17.0,17.0,64000.0,64000.0,Dec-2015,275xx,0.0,0.0,0.0,17.0,17.0,24.0,700.0,704.0,1.0,2.0,37430.0,67.0,0.0,2.0,7386.0,67.0,56300.0,0.0,2.0,0.0,4.0,19159.0,7940.0,77.0,0.0,7.0,11.0,8.0,11.0,17.0,3.0,12.0,17.0,11.0,372109.0,100.0,75.0,75258.0,34500.0,55501.0,36 months,C,C2,Logistics Manager,3 years,MORTGAGE,Not Verified,other,NaN,NC,w,1.0,Individual,N,NaN,NaN,Fully Paid


In [31]:
missing = sampled_by_loan.isna().sum()
missing /= sampled_by_loan.shape[0]
missing *=100
missing = missing.to_frame().rename(columns={0:'Percent Of Missing Values'})
missing

,Percent Of Missing Values
loan_amnt,0.000000
funded_amnt,0.000000
funded_amnt_inv,0.000000
int_rate,0.000000
installment,0.000000
annual_inc,0.000000
annual_inc,0.000000
dti,0.066466
delinq_2yrs,0.000000
delinq_2yrs,0.000000
